# Final Project Check-In

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('.\data\LeagueofLegends.csv')

In [3]:
data.head()

,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,...,redMiddleChamp,goldredMiddle,redADC,redADCChamp,goldredADC,redSupport,redSupportChamp,goldredSupport,redBans,Address
0,NALCS,2015,Spring,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,...",...,Fizz,"[475, 475, 552, 842, 1178, 1378, 1635, 1949, 2...",Sneaky,Sivir,"[475, 475, 532, 762, 1097, 1469, 1726, 2112, 2...",LemonNation,Thresh,"[515, 515, 577, 722, 911, 1042, 1194, 1370, 14...","['Tristana', 'Leblanc', 'Nidalee']",http://matchhistory.na.leagueoflegends.com/en/...
1,NALCS,2015,Spring,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...",...,Azir,"[475, 475, 552, 786, 1097, 1389, 1660, 1955, 2...",CoreJJ,Corki,"[475, 475, 532, 868, 1220, 1445, 1732, 1979, 2...",KiWiKiD,Annie,"[515, 515, 583, 752, 900, 1066, 1236, 1417, 15...","['RekSai', 'Janna', 'Leblanc']",http://matchhistory.na.leagueoflegends.com/en/...
2,NALCS,2015,Spring,Season,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...",...,Azir,"[475, 475, 533, 801, 1006, 1233, 1385, 1720, 1...",Cop,Corki,"[475, 475, 533, 781, 1085, 1398, 1782, 1957, 2...",BunnyFuFuu,Janna,"[515, 515, 584, 721, 858, 1002, 1168, 1303, 14...","['Leblanc', 'Zed', 'RekSai']",http://matchhistory.na.leagueoflegends.com/en/...
3,NALCS,2015,Spring,Season,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...",...,Lulu,"[475, 475, 532, 771, 1046, 1288, 1534, 1776, 2...",KEITH,KogMaw,"[475, 475, 532, 766, 1161, 1438, 1776, 1936, 2...",Xpecial,Janna,"[515, 515, 583, 721, 870, 1059, 1205, 1342, 15...","['RekSai', 'Rumble', 'LeeSin']",http://matchhistory.na.leagueoflegends.com/en/...
4,NALCS,2015,Spring,Season,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20...",...,Lulu,"[475, 475, 532, 807, 1042, 1338, 1646, 1951, 2...",Maplestreet8,Corki,"[475, 475, 532, 792, 1187, 1488, 1832, 2136, 2...",Dodo8,Annie,"[475, 475, 538, 671, 817, 948, 1104, 1240, 136...","['Rumble', 'Sivir', 'Rengar']",http://matchhistory.na.leagueoflegends.com/en/...


In [4]:
positions = ['Top', 'Jungle', 'Middle', 'ADC', 'Support']

regions = data['League'].unique()
years = data['Year'].unique()
seasons = data['Season'].unique()
teams = list(set(data['blueTeamTag'].unique()).union(set(data['redTeamTag'].unique())))


In [5]:
cols = [team + pos + 'Champ' for team in ['blue', 'red'] for pos in positions]
data_condensed = data[['League', 'Year', 'Season', 'blueTeamTag', 'bResult', 'rResult', 'redTeamTag'] + cols]
data_condensed.head()

,League,Year,Season,blueTeamTag,bResult,rResult,redTeamTag,blueTopChamp,blueJungleChamp,blueMiddleChamp,blueADCChamp,blueSupportChamp,redTopChamp,redJungleChamp,redMiddleChamp,redADCChamp,redSupportChamp
0,NALCS,2015,Spring,TSM,1,0,C9,Irelia,RekSai,Ahri,Jinx,Janna,Gnar,Elise,Fizz,Sivir,Thresh
1,NALCS,2015,Spring,CST,0,1,DIG,Gnar,Rengar,Ahri,Caitlyn,Leona,Irelia,JarvanIV,Azir,Corki,Annie
2,NALCS,2015,Spring,WFX,1,0,GV,Renekton,Rengar,Fizz,Sivir,Annie,Sion,LeeSin,Azir,Corki,Janna
3,NALCS,2015,Spring,TIP,0,1,TL,Irelia,JarvanIV,Leblanc,Sivir,Thresh,Gnar,Nunu,Lulu,KogMaw,Janna
4,NALCS,2015,Spring,CLG,1,0,T8,Gnar,JarvanIV,Lissandra,Tristana,Janna,Sion,RekSai,Lulu,Corki,Annie


In [6]:
data_2016 = data_condensed[data_condensed['Year'] == 2016]
data_2016.head()

,League,Year,Season,blueTeamTag,bResult,rResult,redTeamTag,blueTopChamp,blueJungleChamp,blueMiddleChamp,blueADCChamp,blueSupportChamp,redTopChamp,redJungleChamp,redMiddleChamp,redADCChamp,redSupportChamp
245,NALCS,2016,Spring,TSM,0,1,CLG,DrMundo,Elise,TwistedFate,Tristana,Alistar,Jax,RekSai,Lissandra,Kalista,Bard
246,NALCS,2016,Spring,C9,0,1,IMT,Ryze,LeeSin,Ekko,MissFortune,Braum,Chogath,RekSai,TwistedFate,Lucian,Janna
247,NALCS,2016,Spring,DIG,0,1,NRG,Malphite,Kindred,Anivia,Ezreal,Alistar,Fiora,LeeSin,Viktor,Lucian,Janna
248,NALCS,2016,Spring,REN,1,0,TL,DrMundo,RekSai,Orianna,Kalista,Alistar,Gnar,Elise,Viktor,Lucian,Janna
249,NALCS,2016,Spring,FOX,1,0,TIP,Lulu,Elise,Lissandra,Lucian,Thresh,Trundle,Poppy,Viktor,Kindred,Alistar


In [7]:
regional_champ_counts = {}
for region in regions:
    regional_champ_counts[region] = {}
for idx, row in data_2016.iterrows():
    region = row[0]
    for champ in row[7:]:
        if champ in regional_champ_counts[region].keys():
            regional_champ_counts[region][champ] += 1
        else:
            regional_champ_counts[region][champ] = 1

In [8]:
for region, champ_counts in regional_champ_counts.items():
    champ_list = np.array(list(champ_counts.keys()))
    counts = np.array(list(champ_counts.values()))
    idx = np.flip(np.argsort(counts))
    champ_list = champ_list[idx]
    counts = counts[idx]
    
    if len(champ_list) < 1:
        continue
    
    print(region)
    for i in range(min(len(champ_list), 3)):
        print(champ_list[i], counts[i])
    print()

NALCS
RekSai 212
Braum 186
Karma 166

EULCS
Lucian 162
Braum 158
Gragas 147

LCK
Alistar 244
Elise 237
Braum 200

LMS
Braum 125
Trundle 119
Gragas 119

CBLoL
Elise 79
Braum 74
Lucian 73

TCL
Trundle 109
Lucian 108
Elise 101

OPL
Braum 106
RekSai 100
Lucian 95

LJL
Braum 59
Lucian 52
Alistar 50

LCL
Lucian 72
RekSai 56
Elise 55

WC
Karma 42
Jhin 39
LeeSin 37

MSI
Poppy 33
Ekko 33
Lucian 30

IEM
Poppy 37
LeeSin 37
Ezreal 33

